In [1]:
# CNN Indonesia
# Kompas
# Kontan
# liputan6
# tribunews
# the jakarta post

In [1]:
from bs4 import BeautifulSoup
from googletrans import Translator; translator = Translator();
from tqdm import tqdm
from urllib.request import urlopen, Request
from urllib.parse import urlparse

import requests
import datefinder
import dateutil.parser
import datetime
import time

In [2]:
month_id = ["Januari", "Februari", "Maret", "April", "Mei", 
            "Juni", "Juli", "Agustus", "September", "Oktober", 
            "November", "Desember"]
month_dict = {month: translator.translate(month, src="id", dest="en").text for month in month_id}
month_dict_shrt= {month[:3]: translator.translate(month, src="id", dest="en").text[:3] for month in month_id}

In [3]:
def get_date_year(url, html_tag, attribute, element):
    assert isinstance(url, str), "Argument given not a str, but a {}".format(type(url))
    MAX_RETRIES = 20
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(max_retries=MAX_RETRIES)
    session.mount('https://', adapter)
    session.mount('http://', adapter)

    r = session.get(url, headers=headers)
    
    month_dict = {'Januari': 'January',
                 'Februari': 'February',
                 'Maret': 'March',
                 'April': 'April',
                 'Mei': 'May',
                 'Juni': 'June',
                 'Juli': 'July',  
                 'Agustus': 'August',
                 'September': 'September',
                 'Oktober': 'October',
                 'November': 'November',
                 'Desember': 'December'}
    
    month_dict_shrt = {'Jan': 'Jan',
                         'Feb': 'Feb',
                         'Mar': 'Mar',
                         'Apr': 'Apr',
                         'Mei': 'May',
                         'Jun': 'Jun',
                         'Jul': 'Jul',
                         'Agu': 'Aug',
                         'Sep': 'Sep',
                         'Okt': 'Oct',
                         'Nov': 'Nov',
                         'Des': 'Dec'}
    

    soup = BeautifulSoup(r.content)
    
    txt = soup.find(html_tag, {attribute: element}).text
    
    for month in month_dict.keys():
    
        if month in txt:
            txt = txt.replace(month, month_dict[month])
        else:
            pass

    for month in month_dict_shrt.keys():

        if month in txt:
            txt = txt.replace(month, month_dict_shrt[month])
        else:
            pass
#     txt = translator.translate(txt, src="id", dest="en")
    print(txt)
    match = dateutil.parser.parse(txt, fuzzy=True).year
    return match
    
#     try:
#         matches = datefinder.find_dates(txt)
#     else:
#         raise 

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import time

In [5]:
from selenium.webdriver.chrome.options import Options
options = Options()
options.headless = True

# Detik Section

In [7]:
chromedriver_path = "/mnt/c/Users/Gilang R Ilhami/Desktop/personal_projects/ibm_stuff/chromedriver.exe"
driver = webdriver.Chrome(executable_path=chromedriver_path, options=options)

In [8]:
def detik_scraper(driver, first_year, second_year, keyword):
    driver.get("https://www.detik.com/")
    driver.implicitly_wait(20)

    driver.implicitly_wait(10)
    input_search = driver.find_element_by_xpath("//*[@id='search_navbar']/input[1]")
    input_search.send_keys(keyword.lower())
    input_search.send_keys(Keys.RETURN)
    
    driver.implicitly_wait(10)
    news_list = driver.find_element_by_xpath("/html/body/div[2]/div/div[2]")
    link_list = news_list.find_elements_by_tag_name("a")
    link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]
    
    if bool(link_list) is False:
        return None

    last_idx = -1
    for _ in range(len(link_list)):
        print("Last Index", last_idx)
        try:
            last_news_year = get_date_year(link_list[last_idx], "div", "class", "date")
            break
        except Exception as e:
            print(e)
        last_idx -= 1
        
    if last_news_year < first_year:
        all_link_list = link_list
    else:
        all_link_list = []
        
    while last_news_year >= first_year:
        print("Getting links...")

        # Wait to load page
        time.sleep(5)

        driver.implicitly_wait(10)
        news_list = driver.find_element_by_xpath("/html/body/div[2]/div/div[2]")
        link_list = news_list.find_elements_by_tag_name("a")
        link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]

        all_link_list.extend(link_list)

        last_idx = -1

        for _ in range(len(link_list)):
            try:
                last_news_year = get_date_year(link_list[last_idx], "div", "class", "date")
                break
            except Exception as e:
                print(e)
            last_idx -= 1
        
        try:
            page_list = driver.find_element_by_tag_name("div[class*='paging text_center']")
            current_page = page_list.find_element_by_tag_name("a[class*='selected']")
            current_page_num = int(current_page.text)
            current_page_link = current_page.get_attribute("href")
        except Exception as e:
            print("Something went wrong while going to next page.")
            print(e)
            
            print("Going back")
            driver.execute_script("window.history.go(-1)")
            
            print("Skipping next page")
            next_next_page = current_page_num + 2
            next_next_link = current_page_link[:-1] + str(next_next_page)
            driver.get(next_next_link)
        else:
            next_page = page_list.find_elements_by_tag_name("a")[-1]
            next_page.click()
        
    link_list = all_link_list.copy()
    first_year_contents = []
    second_year_contetns = []

    all_index = set(map(link_list.index, link_list))

    error_dates_index = set()

    for link in link_list:
        try:
            date = get_date_year(link,"div", "class", "date")
        except Exception as e:
            print(link)
            print(e, end="\n\n")
            error_idx = link_list.index(link)
            error_dates_index.add(error_idx)

    can_get_dates_index = list(all_index - error_dates_index)

    current_date = datetime.datetime.now()
    current_year = current_date.year

    for content_index in can_get_dates_index:

        # News Publish Date
        content_date = get_date_year(link_list[content_index],"div", "class", "date")
        print(content_date)

        # First year news
        if content_date >= first_year and content_date < second_year:
            first_year_contents.append(link_list[content_index])
        elif content_date >= second_year and content_date < current_year:
            second_year_contetns.append(link_list[content_index])
        else:
            pass
        
    return first_year_contents, second_year_contetns

In [10]:
urls_detik = detik_scraper(driver, 2017, 2018, "tenaga nuklir")

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=57475): Max retries exceeded with url: /session/bf295679a0146ff67f06dffec3084e7a/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fad7f158400>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [38]:
urls_detik[1]

['https://finance.detik.com/foto-bisnis/d-4323338/adhi-karya-peroleh-kontrak-baru-rp-126-t-hingga-november-2018',
 'https://finance.detik.com/foto-bisnis/d-4227265/anak-usaha-adhi-karya-bikin-apartemen-tod',
 'https://foto.detik.com/readfoto/2018/09/19/100014/4218560/464/1/ini-dia-rumah-transisi-untuk-korban-gempa-lombok',
 'https://finance.detik.com/infrastruktur/d-4365614/kereta-lrt-jabodebek-uji-coba-cawang-cibubur-april-2019',
 'https://finance.detik.com/berita-ekonomi-bisnis/d-4365554/22-bumn-bentuk-yayasan-sosial',
 'https://finance.detik.com/infrastruktur/d-4365549/hingga-akhir-2018-proyek-lrt-jabodebek-sudah-setengah-jadi',
 'https://finance.detik.com/berita-ekonomi-bisnis/d-4365342/jokowi-teken-aturan-holding-infrastruktur-awal-2019',
 'https://finance.detik.com/loker/d-4360523/buruan-lamar-adhi-karya-buka-lowongan--buat-yang-baru-lulus',
 'https://finance.detik.com/infrastruktur/d-4359942/alat-berat-dikerahkan-tanggulangi-dampak-tsunami-selat-sunda',
 'https://travel.detik.co

# Liputan 6 Section

In [12]:
chromedriver_path = "/mnt/c/Users/Gilang R Ilhami/Desktop/personal_projects/ibm_stuff/chromedriver.exe"
driver = webdriver.Chrome(executable_path=chromedriver_path)

In [6]:
def liputan6_scraper(driver, first_year, second_year, keyword):
    driver.get("https:///www.liputan6.com/")
    driver.implicitly_wait(20)
    
    driver.implicitly_wait(10)
    input_search = driver.find_element_by_xpath("//*[@id='q']")
    input_search.send_keys(keyword.lower())
    input_search.send_keys(Keys.RETURN)
    
    driver.implicitly_wait(10)
    news_list = driver.find_element_by_xpath("//*[@id='main']/div[3]/div/article/div/section")
    link_list = news_list.find_elements_by_tag_name("h4")
    link_list = [title.find_element_by_tag_name("a") for title in link_list]
    link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]
    
    if bool(link_list) is False:
        return None
    
    last_idx = -1
    for _ in range(len(link_list)):
        print("Last Index", last_idx)
        try:
            last_news_year = get_date_year(link_list[last_idx], "time", "class", "read-page--header--author__datetime updated")
            break
        except Exception as e:
            print(e)
        last_idx -= 1
        
    
    page_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    # Sroll to the desired number of contents
    while last_news_year >= first_year:
        print("Scrolling down")
        # Wait to load page
        time.sleep(5)
            
        driver.implicitly_wait(10)
        news_list = driver.find_element_by_xpath("//*[@id='main']/div[3]/div/article/div/section")
        link_list = news_list.find_elements_by_tag_name("h4")
        link_list = [title.find_element_by_tag_name("a") for title in link_list]
        link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]
        
        last_idx = -1
        
        for _ in range(len(link_list)):
            try:
                last_news_year = get_date_year(link_list[last_idx],"time", "class", "read-page--header--author__datetime updated")
                break
            except:
                pass
            last_idx -= 1
    driver.implicitly_wait(10)
    news_list = driver.find_element_by_xpath("//*[@id='main']/div[3]/div/article/div/section")
    link_list = news_list.find_elements_by_tag_name("h4")
    link_list = [title.find_element_by_tag_name("a") for title in link_list]
    link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]
                
#     driver.close()
#     return link_list
        
    first_year_contents = []
    second_year_contetns = []
    
    all_index = set(map(link_list.index, link_list))
    
    error_dates_index = set()
    
    for link in link_list:
        try:
            date = get_date_year(link,"time", "class", "read-page--header--author__datetime updated")
        except Exception as e:
            print(link)
            print(e, end="\n\n")
            error_idx = link_list.index(link)
            error_dates_index.add(error_idx)
            
    can_get_dates_index = list(all_index - error_dates_index)
    
    current_date = datetime.datetime.now()
    current_year = current_date.year
    
    for content_index in can_get_dates_index:
        
        # News Publish Date
        content_date = get_date_year(link_list[content_index],"time", "class", "read-page--header--author__datetime updated")
        print(content_date)
        
        # First year news
        if content_date >= first_year and content_date < second_year:
            first_year_contents.append(link_list[content_index])
        elif content_date >= second_year and content_date < current_year:
            second_year_contetns.append(link_list[content_index])
        else:
            pass
            
    return first_year_contents, second_year_contetns


In [46]:
urls_liputan6 = liputan6_scraper(driver, 2017, 2018, "rv;ewo;goaergbr")

In [48]:
urls_liputan6

In [ ]:
urls_liputan6

In [35]:
liputan6_scraped = False

while liputan6_scraped is False:
    try:
        urls_liputan6 = liputan6_scraper(driver)
        liputan6_scraped = True
    except Exception as e:
        print(e)
        pass

KeyboardInterrupt: 

In [5]:
urls_liputan6

['https://www.liputan6.com/bisnis/read/3481756/dalam-3-bulan-penyaluran-kredit-bank-bca-tembus-rp-470-triliun?source=search',
 'https://www.liputan6.com/news/read/3407171/ruko-bank-bca-melawai-terbakar-ini-penyebabnya?source=search',
 'https://www.liputan6.com/photo/read/2450639/laba-bank-bca-tumbuh-93-pada-2015?source=search',
 'https://www.liputan6.com/bisnis/read/2107040/biaya-transaksi-via-atm-bank-bca-tak-naik?source=search',
 'https://www.liputan6.com/bisnis/read/2097772/bunga-deposito-bank-bca-turun-05-awal-september?source=search',
 'https://www.liputan6.com/buser/read/404243/nasabah-bank-bca-bekasi-dirampok?source=search',
 'https://www.liputan6.com/video/read/637873/nasabah-bank-bca-bekasi-dirampok?source=search',
 'https://www.liputan6.com/buser/read/359788/pembunuh-satpam-bank-bca-ditangkap?source=search',
 'https://www.liputan6.com/video/read/653332/pembunuh-satpam-bank-bca-ditangkap?source=search',
 'https://www.liputan6.com/video/read/653668/perampokan-di-bank-bca-kopo-t

# Kompas Section

In [6]:
chromedriver_path = "/mnt/c/Users/Gilang R Ilhami/Desktop/personal_projects/ibm_stuff/chromedriver.exe"
driver = webdriver.Chrome(executable_path=chromedriver_path)

In [7]:
def kompas_scraper(driver, first_year, second_year, keyword):
    driver.get("https://www.kompas.com/")
    
    driver.implicitly_wait(10)
    input_search = driver.find_element_by_xpath("//*[@id='search']")
    input_search.send_keys(keyword.lower())
    input_search.send_keys(Keys.RETURN)
    
#     sort_options = driver.find_element_by_xpath('//*[@id="___gcse_0"]/div/div/div/div[3]/table/tbody/tr/td[2]/div/div[2]/div[1]/div[1]')
#     sort_options.click()
    
#     select_date = driver.find_element_by_xpath('//*[@id="___gcse_0"]/div/div/div/div[3]/table/tbody/tr/td[2]/div/div[2]/div[2]/div[2]/div')
#     select_date.click()
    try:
        pagination_element = driver.find_element_by_tag_name("div[class*='gsc-cursor']")
        pages = pagination_element.find_elements_by_tag_name("div[class*='gsc-cursor-page']")
    except Exception as e:
        print("No pagination element")
        time.sleep(5)
        news_list = driver.find_element_by_xpath("//*[@id='___gcse_0']/div/div/div/div[5]/div[2]/div/div")
        link_list = news_list.find_elements_by_tag_name("a[class*='gs-title']")
        time.sleep(5)
        link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]
        
        pages = None
        
        if bool(link_list) is False:
            return None
        else:
            all_link_list = link_list
    
    if pages is not None:
        all_links_list = []
        for page_idx in tqdm(range(1, len(pages) + 1)):

            if page_idx != 1:
                # Handling element when not loaded
                cannot_click = True
                attempt = 0

                # While element is not clicked
                while cannot_click:
                    try:
                        print("Looking for element.")
                        element_xpath = f"//*[@id='___gcse_0']/div/div/div/div[5]/div[2]/div/div/div[2]/div[11]/div/div[{page_idx}]"

                        # Wait untill element exists
                        element = WebDriverWait(driver, 10).until(
                                lambda driver : driver.find_element_by_xpath(element_xpath)
                        )

                        # Get element 
                        next_page = driver.find_element_by_xpath(element_xpath)

                    except Exception as element_error:

                        print("Element checking failed. Something went wrong.\n", element_error)

                        attempt += 1

                        print(f"Element failed. Repeating process. Currently attempt {attempt}")

                    else:
                        print("Element found. Waiting to be clickable.")
                        try:
                            clickable = WebDriverWait(driver, 10).until(
                                EC.element_to_be_clickable((By.XPATH, element_xpath))
                            )
                            next_page.click()
                            cannot_click = False
                        except Exception as clickable_error:
                            print("Clickalbe checking failed. Something went wrong.\n", clickable_error, end ="\n\n")
                            attempt += 1
                            print(f"Clicking failed. Repeating process. Currently attempt {attempt}")

            time.sleep(5)
            news_list = driver.find_element_by_xpath("//*[@id='___gcse_0']/div/div/div/div[5]/div[2]/div/div")
            link_list = news_list.find_elements_by_tag_name("a[class*='gs-title']")
            time.sleep(5)
            link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]

            urls = list(set(link_list))
            urls = list(filter(None, urls))

            all_links_list.extend(urls)

            pagination_check = driver.find_element_by_tag_name("div[class*='gsc-cursor']")
            pagination_check = pagination_check.find_elements_by_tag_name("div[class*='gsc-cursor-page']")
            if page_idx == len(pagination_check):
                break
    
    link_list = all_links_list
    first_year_contents = []
    second_year_contetns = []
    
    all_index = set(map(link_list.index, link_list))
    
    error_dates_index = set()
    
    for link in tqdm(link_list):
        try:
            date = get_date_year(link,"div", "class", "read__time")
        except Exception as e:
            print(link)
            print(e, end="\n\n")
            error_idx = link_list.index(link)
            error_dates_index.add(error_idx)
            
    can_get_dates_index = list(all_index - error_dates_index)
    
    current_date = datetime.datetime.now()
    current_year = current_date.year
    
    for content_index in tqdm(can_get_dates_index):
        
        # News Publish Date
        content_date = get_date_year(link_list[content_index],"div", "class", "read__time")
        print(content_date)
        
        # First year news
        if content_date >= first_year and content_date < second_year:
            first_year_contents.append(link_list[content_index])
        elif content_date >= second_year and content_date < current_year:
            second_year_contetns.append(link_list[content_index])
        else:
            pass
    
    
    return first_year_contents, second_year_contetns

In [64]:
urls_kompas = kompas_scraper(driver, 2017, 2018, "Sawit Sumbermas Sarana")









  0%|          | 0/10 [00:00<?, ?it/s]







 10%|█         | 1/10 [00:10<01:34, 10.53s/it]

Looking for element.
Element found. Waiting to be clickable.










 20%|██        | 2/10 [00:21<01:25, 10.72s/it]

Looking for element.
Element found. Waiting to be clickable.










 30%|███       | 3/10 [00:32<01:15, 10.83s/it]

...

Kompas.com - 23/10/2015, 11:52 WIB
2015











 97%|█████████▋| 33/34 [01:14<00:02,  2.07s/it]

Kompas.com - 09/11/2009, 20:03 WIB
2009











100%|██████████| 34/34 [01:16<00:00,  1.97s/it]

Kompas.com - 22/06/2017, 07:30 WIB
2017


In [66]:
assert urls_kompas is not None

In [67]:
urls_kompas

(['https://ekonomi.kompas.com/read/2017/12/28/144303026/bangun-pabrik-cpo-sawit-sumbermas-anggarkan-rp-270-miliar',
  'https://ekonomi.kompas.com/read/2017/12/28/131500426/hingga-september-2017-sawit-sumbermas-catat-penjualan-rp-2-4-triliun',
  'https://ekonomi.kompas.com/read/2017/12/28/181200926/bayar-utang-sawit-sumbermas-akan-terbitkan-global-bond-di-singapura',
  'https://sains.kompas.com/read/2017/01/10/203056226/read-brandzview.html',
  'https://regional.kompas.com/read/2017/05/22/16192851/dilantik.bupati.kotawaringin.barat.jadi.bupati.perempuan.pertama.di.kalteng',
  'https://regional.kompas.com/read/2017/06/22/07300011/dilengserkan.karena.selingkuh.mantan.bupati.katingan.tetap.eksis.di.politik'],
 ['https://ekonomi.kompas.com/read/2018/06/07/173809626/ditutup-menguat-ihsg-kembali-tembus-level-6100',
  'https://pemilu.kompas.com/read/2018/07/03/134230526/ihsg-ditutup-melemah-saham-pgas-merosot-tajam',
  'https://ekonomi.kompas.com/read/2018/08/20/124224926/sesi-i-ihsg-ditutup-m

In [7]:
kompas_scraped = False

while kompas_scraped is False:
    try:
        urls_kompas = kompas_scraper(driver)
        kompas_scraped = True
    except:
        pass

In [8]:
urls_kompas

['https://money.kompas.com/read/2019/07/17/183500126/bca--perubahan-zaman-dulu-buka-cabang-40-sekarang-kurang-dari-20',
 'https://money.kompas.com/read/2019/07/17/183500126/bca--perubahan-zaman-dulu-buka-cabang-40-sekarang-kurang-dari-20',
 'https://money.kompas.com/read/2019/06/30/124400426/bca-raih-penghargaan-bank-terbaik-di-indonesia-dan-asia',
 'https://money.kompas.com/read/2019/06/30/124400426/bca-raih-penghargaan-bank-terbaik-di-indonesia-dan-asia',
 'https://money.kompas.com/read/2019/05/24/182022426/amankah-penggunaan-vpn-untuk-layanan-perbankan-ponsel-ini-kata-bank-bca',
 'https://money.kompas.com/read/2019/05/24/182022426/amankah-penggunaan-vpn-untuk-layanan-perbankan-ponsel-ini-kata-bank-bca',
 'https://money.kompas.com/read/2019/05/20/101200826/cara-daftar-mobile-banking-bca?page=all',
 'https://money.kompas.com/read/2019/05/20/101200826/cara-daftar-mobile-banking-bca?page=all',
 'https://ekonomi.kompas.com/read/2018/12/24/174639826/catat-ini-jadwal-operasional-bca-selama

# Kontan Section

In [9]:
def kontan_scraper(driver):
    driver.get("https://www.kontan.co.id/")
    
    driver.implicitly_wait(10)
    input_search = driver.find_element_by_xpath("//*[@id='wrapforleftpush']/header/div[1]/div[2]/div/div[2]/form/div/input")
    input_search.send_keys("bank bca")
    input_search.send_keys(Keys.RETURN)
    
    driver.implicitly_wait(10)
    news_list = driver.find_elements_by_tag_name("h1")
    link_list = [title.find_element_by_tag_name("a") for title in news_list]
    link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]

    return link_list

In [18]:

chromedriver_path = "/mnt/c/Users/Gilang R Ilhami/Desktop/personal_projects/ibm_stuff/chromedriver.exe"
driver = webdriver.Chrome(executable_path=chromedriver_path)

In [19]:
driver.get("https://www.kontan.co.id/")
    
driver.implicitly_wait(10)
input_search = driver.find_element_by_xpath("//*[@id='wrapforleftpush']/header/div[1]/div[2]/div/div[2]/form/div/input")
input_search.send_keys("bank indonesia".lower())
input_search.send_keys(Keys.RETURN)



In [ ]:
driver.implicitly_wait(10)
news_list = driver.find_elements_by_tag_name("h1")
link_list = [title.find_element_by_tag_name("a") for title in news_list]
link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]

all_links_list.extend(link_list)

In [20]:
paging = driver.find_element_by_xpath("//*[@id='content_news']/div[2]")
next_page = paging[-1]
next_page.click()

TypeError: 'WebElement' object does not support indexing

In [21]:
next_page = paging.find_elements_by_tag_name("a")

In [27]:
e_text, = [e  for e in next_page if e.text == "Next"]
e_text.click()

In [17]:
end_of_page = False
all_links_list = []
while not end_of_page:
    
    time.sleep(5)
    driver.implicitly_wait(10)
    news_list = driver.find_elements_by_tag_name("h1")
    link_list = [title.find_element_by_tag_name("a") for title in news_list]
    link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]
    all_links_list.extend(link_list)
    
    try:
        next_page = driver.find_element_by_xpath("//*[@id='content_news']/div[2]/a[1]")
        next_page.click()
        print("next page element found")
    except Exception as e:
        end_of_page = True
        print(e)
        break

next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found
next page element found


KeyboardInterrupt: 

In [18]:
all_links_list

[]

In [10]:
kontan_scraped = False

while kontan_scraped is False:
    try:
        urls_kontan = kontan_scraper(driver)
        kontan_scraped = True
    except:
        pass

In [11]:
urls_kontan

['https://keuangan.kontan.co.id/news/begini-cara-bca-menyiapkan-bank-royal-untuk-bersaing-dengan-fintech-lending',
 'https://insight.kontan.co.id/news/tiga-bank-besar-mencetak-untung-gede-laba-bersih-bca-naik-paling-kencang',
 'https://keuangan.kontan.co.id/news/di-antara-tiga-bank-besar-laba-bank-bca-tumbuh-paling-tinggi',
 'https://keuangan.kontan.co.id/news/antisipasi-persaingan-dengan-fintech-bca-pastikan-bank-royal-jadi-bank-digital',
 'https://keuangan.kontan.co.id/news/deposito-bank-bca-tumbuh-181-di-semester-i-apa-dampaknya-terhadap-kinerja',
 'https://keuangan.kontan.co.id/news/naik-126-laba-bank-bca-bbca-tembus-rp-129-triliun-di-semester-i-2019',
 'https://keuangan.kontan.co.id/news/begini-strategi-bank-bca-hadapi-sektor-multifinance-yang-melandai',
 'https://keuangan.kontan.co.id/news/intip-kurs-jual-dollar-as-terhadap-rupiah-di-bca-bank-mandiri-bni-selasa-167',
 'https://keuangan.kontan.co.id/news/rupiah-menguat-ini-kurs-jual-dollar-di-bca-bni-dan-bank-mandiri-senin-pagi-15

# CNN 6 Section

In [18]:

chromedriver_path = "/mnt/c/Users/Gilang R Ilhami/Desktop/personal_projects/ibm_stuff/chromedriver.exe"
driver = webdriver.Chrome(executable_path=chromedriver_path)

In [8]:
def cnnindo_scraper(driver, first_year, second_year, keyword):
    driver.get("https://www.cnnindonesia.com")

    button = driver.find_element_by_xpath("//*[@id='nav_search']/a")
    driver.implicitly_wait(10)
    button.click()

    driver.implicitly_wait(10)
    input_search = driver.find_element_by_xpath("//*[@id='search_form']/input")
    input_search.send_keys(keyword.lower())
    input_search.send_keys(Keys.RETURN)
    
    driver.implicitly_wait(10)
    news_list = driver.find_elements_by_tag_name("article")
    link_list = [title.find_element_by_tag_name("a") for title in news_list]
    link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]
    
    if bool(link_list) is False:
            return None
    
    last_idx = -1
    for _ in range(len(link_list)):
        print("Last Index", last_idx)
        try:
            last_news_year = get_date_year(link_list[last_idx], "div", "class", "date")
            break
        except Exception as e:
            print(e)
        last_idx -= 1

    print(last_news_year)
    page_height = driver.execute_script("return document.documentElement.scrollHeight")

    scrolling = 0

    # Sroll to the desired number of contents
    while last_news_year >= first_year:
        print("Scrolling down")

        # Handling element when not loaded
        cannot_click = True
        attempt = 0

        # While element is not clicked
        while cannot_click:
            try:
                print("Looking for element.")
                element_xpath = "//*[@id='content']/div/div[2]/div/div[2]/a"

                # Wait untill element exists
                element = WebDriverWait(driver, 10).until(
                        lambda driver : driver.find_element_by_xpath(element_xpath)
                )

                # Get element 
                next_page = driver.find_element_by_xpath(element_xpath)

            except Exception as element_error:

                print("Element checking failed. Something went wrong.\n", element_error)

                attempt += 1

                print(f"Element failed. Repeating process. Currently attempt {attempt}")

            else:
                print("Element found. Waiting to be clickable.")
                try:
                    clickable = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, element_xpath))
                    )
                    next_page.click()
                    cannot_click = False
                except Exception as clickable_error:
                    print("Clickalbe checking failed. Something went wrong.\n", clickable_error, end ="\n\n")
                    attempt += 1
                    print(f"Clicking failed. Repeating process. Currently attempt {attempt}")


        # Wait to load page
        time.sleep(5)

        # Calculate new scroll height and compare with last scroll height
#         new_height = driver.execute_script("return document.documentElement.scrollHeight")
#         if new_height == page_height:
#             print("break")
#             break
#         page_height = new_height

        driver.implicitly_wait(10)
        news_list = driver.find_elements_by_tag_name("article")
        link_list = [title.find_element_by_tag_name("a") for title in news_list]
        link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]

        last_idx = -1

        for _ in range(len(link_list)):
            try:
                last_news_year = get_date_year(link_list[last_idx],"div", "class", "date")
                print(last_news_year)
                break
            except Exception as e:
                print(e)
                pass
            last_idx -= 1
        scrolling += 1
    driver.implicitly_wait(10)
    news_list = driver.find_elements_by_tag_name("article")
    link_list = [title.find_element_by_tag_name("a") for title in news_list]
    link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]
 
    first_year_contents = []
    second_year_contetns = []

    all_index = set(map(link_list.index, link_list))

    error_dates_index = set()

    for link in tqdm(link_list):
        try:
            date = get_date_year(link,"div", "class", "date")
        except Exception as e:
            print(link)
            print(e, end="\n\n")
            error_idx = link_list.index(link)
            error_dates_index.add(error_idx)

    can_get_dates_index = list(all_index - error_dates_index)

    current_date = datetime.datetime.now()
    current_year = current_date.year

    for content_index in tqdm(can_get_dates_index):

        # News Publish Date
        content_date = get_date_year(link_list[content_index],"div", "class", "date")
        print(content_date)

        # First year news
        if content_date >= first_year and content_date < second_year:
            first_year_contents.append(link_list[content_index])
        elif content_date >= second_year and content_date < current_year:
            second_year_contetns.append(link_list[content_index])
        else:
            pass
    return first_year_contents, second_year_contetns

In [24]:
urls_cnnindo = cnnindo_scraper(driver, 2017, 2018, "Sawit Sumbermas Sarana")

Last Index -1

Dinda Audriene Muthmainah,  CNN Indonesia | Senin, 09/10/2017 09:55 WIB
                
2017
Scrolling down
Looking for element.
Element found. Waiting to be clickable.
Clickalbe checking failed. Something went wrong.
 Message: element click intercepted: Element <a href="https://www.cnnindonesia.com/search/?query=sawit sumbermas sarana&amp;kanal=&amp;date=Semua Tgl" class="loading btn gtm_button_indeks" d:query="sawit sumbermas sarana" d:mode="click" d:max="1000">...</a> is not clickable at point (384, 207). Other element would receive the click: <div class="container clearfix">...</div>
  (Session info: chrome=76.0.3809.100)


Clicking failed. Repeating process. Currently attempt 1
Looking for element.
Element found. Waiting to be clickable.

Immanuel Giras Pasopati,  CNN Indonesia | Jumat, 16/10/2015 17:43 WIB
                
2015


  6%|▌         | 2/35 [00:00<00:04,  7.90it/s]


                     CNN Indonesia | Kamis, 15/08/2019 23:37 WIB
                

                     CNN Indonesia | Kamis, 15/08/2019 23:07 WIB
                


  9%|▊         | 3/35 [00:00<00:04,  7.97it/s]


                     CNN Indonesia | Kamis, 15/08/2019 22:50 WIB
                


...


​Elisa Valenta Sari,  CNN Indonesia | Minggu, 15/01/2017 17:03 WIB
                
2017

Safyra Primadhyta,  CNN Indonesia | Rabu, 17/02/2016 19:07 WIB
                
2016


 97%|█████████▋| 34/35 [00:04<00:00,  8.31it/s]


Giras Pasopati,  CNN Indonesia | Senin, 25/01/2016 17:30 WIB
                
2016

Antara & Augst Supriadi,  CNN Indonesia | Selasa, 10/11/2015 17:45 WIB
                
2015


100%|██████████| 35/35 [00:05<00:00,  8.51it/s]


Immanuel Giras Pasopati,  CNN Indonesia | Jumat, 16/10/2015 17:43 WIB
                
2015


In [25]:
urls_cnnindo

(['https://www.cnnindonesia.com/ekonomi/20171024075752-92-250512/terpompa-sentimen-positif-ihsg-bertahan-di-zona-hijau',
  'https://www.cnnindonesia.com/ekonomi/20171009083559-92-247054/sentimen-positif-sentuh-saham-cpo-dan-perbankan',
  'https://www.cnnindonesia.com/ekonomi/20170612091716-92-221025/pasokan-cpo-melonjak-saham-agrikultur-melorot',
  'https://www.cnnindonesia.com/ekonomi/20170602091414-78-218813/enam-bank-disebut-ikut-andil-rusak-hutan',
  'https://www.cnnindonesia.com/ekonomi/20170115145011-78-186417/bni-kucurkan-kredit-rp6-t-ke-perusahaan-sawit-di-kalteng'],
 ['https://www.cnnindonesia.com/ekonomi/20181213161658-92-353458/bos-blue-bird-ditendang-dari-daftar-50-orang-terkaya-ri',
  'https://www.cnnindonesia.com/ekonomi/20181004141906-92-335708/harga-sawit-anjlok-moodys-sebut-peringkat-produsen-berisiko'])

In [13]:
cnnindo_scraped = False

while cnnindo_scraped is False:
    try:
        urls_cnnindo = cnnindo_scraper(driver)
        cnnindo_scraped = True
    except:
        pass

# TribunNews Section

In [93]:

chromedriver_path = "/mnt/c/Users/Gilang R Ilhami/Desktop/personal_projects/ibm_stuff/chromedriver.exe"
driver = webdriver.Chrome(executable_path=chromedriver_path)

In [9]:
def tribunnews_scraper(driver, first_year, second_year, keyword):
    driver.get("https://www.tribunnews.com/")

    button = driver.find_element_by_xpath("/html/body/div[2]/div/ul/li[2]")
    button.click()

    input_search = driver.find_element_by_xpath("//*[@id='cse-search-box']/input[1]")
    input_search.send_keys(keyword.lower())
    input_search.send_keys(Keys.RETURN)
    
    
    try:
        pagination_element = driver.find_element_by_tag_name("div[class*='gsc-cursor']")
        pages = pagination_element.find_elements_by_tag_name("div[class*='gsc-cursor-page']")
    except Exception as e:
        print("No pagination element")
        
        time.sleep(5)
        news_list = driver.find_element_by_xpath("//*[@id='___gcse_0']/div/div/div/div[5]/div[2]/div[2]/div")
        link_list = news_list.find_elements_by_tag_name("a[class*='gs-title']")
        time.sleep(5)
        link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]
        
        pages = None
        
        if bool(link_list) is False:
            return None
        else:
            all_link_list = link_list
    
    if pages is not None:
        all_link_list = []
        for page_idx in tqdm(range(1, len(pages) + 1)):

            # If index is not 1, go to that page of the index
            if page_idx != 1:

                # Handling element when not loaded
                cannot_click = True
                attempt = 0

                # While element is not clicked
                while cannot_click:
                    try:
                        print("Looking for element.")
                        element_xpath = f"//*[@id='___gcse_0']/div/div/div/div[5]/div[2]/div[2]/div/div[2]/div[11]/div/div[{page_idx}]"

                        # Wait untill element exists
                        element = WebDriverWait(driver, 10).until(
                                lambda driver : driver.find_element_by_xpath(element_xpath)
                        )

                        # Get element 
                        next_page = driver.find_element_by_xpath(element_xpath)

                    except Exception as element_error:

                        print("Element checking failed. Something went wrong.\n", element_error)

                        attempt += 1

                        print(f"Element failed. Repeating process. Currently attempt {attempt}")

                    else:
                        print("Element found. Waiting to be clickable.")
                        try:
                            clickable = WebDriverWait(driver, 10).until(
                                EC.element_to_be_clickable((By.XPATH, element_xpath))
                            )
                            next_page.click()
                            cannot_click = False
                        except Exception as clickable_error:
                            print("Clickalbe checking failed. Something went wrong.\n", clickable_error, end ="\n\n")
                            attempt += 1
                            print(f"Clicking failed. Repeating process. Currently attempt {attempt}")



            time.sleep(5)
            news_list = driver.find_element_by_xpath("//*[@id='___gcse_0']/div/div/div/div[5]/div[2]/div[2]/div")
            link_list = news_list.find_elements_by_tag_name("a[class*='gs-title']")
            time.sleep(5)
            link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]

            # Remove duplicates and none values
            urls = list(set(link_list))
            urls = list(filter(None, urls))

            all_link_list.extend(urls)

            pagination_check = driver.find_element_by_tag_name("div[class*='gsc-cursor']")
            pagination_check = pagination_check.find_elements_by_tag_name("div[class*='gsc-cursor-page']")
            if page_idx == len(pagination_check):
                break
            
    link_list = all_link_list.copy()

    first_year = 2017
    second_year = 2018

    first_year_contents = []
    second_year_contents = []

    all_index = set(map(link_list.index, link_list))

    error_dates_index = set()

    for link in tqdm(link_list):
        try:
            date = get_date_year(link,"time", "class", "grey")
        except Exception as e:
            print(link)
            print(e, end="\n\n")
            error_idx = link_list.index(link)
            error_dates_index.add(error_idx)

    can_get_dates_index = list(all_index - error_dates_index)

    current_date = datetime.datetime.now()
    current_year = current_date.year

    for content_index in tqdm(can_get_dates_index):

        # News Publish Date
        content_date = get_date_year(link_list[content_index],"time", "class", "grey")
        print(content_date)

        # First year news
        if content_date >= first_year and content_date < second_year:
            first_year_contents.append(link_list[content_index])
        elif content_date >= second_year and content_date < current_year:
            second_year_contents.append(link_list[content_index])
        else:
            pass
    
    return first_year_contents, second_year_contents

In [72]:
urls_tribunnews = tribunnews_scraper(driver, 2017, 2018, "rv;ewo;goaergbr")

No pagination element


In [74]:
type(urls_tribunnews)

NoneType

In [69]:
urls_tribunnews = first_year_contents, second_year_contetns
urls_tribunnews

(['https://www.tribunnews.com/bisnis/2017/12/08/dana-subsidi-lrt-selama-12-tahun-diambil-dari-apbn',
  'https://www.tribunnews.com/metropolitan/2017/10/13/adhi-karya-bantah-tiang-lrt-bikin-ambles-di-depan-menara-saidah',
  'https://www.tribunnews.com/bisnis/2017/12/12/apartmen-lrt-superblok-the-conexio-di-cikunir-dibanderol-mulai-rp-400-juta',
  'https://www.tribunnews.com/bisnis/2017/12/07/ambruknya-saham-saham-emiten-bumn-konstruksi-di-lantai-bursa',
  'https://www.tribunnews.com/bisnis/2017/12/08/investasi-lrt-ditekan-jadi-rp-299-triliun-dibiayai-rights-issue-adhi-karya-dan-pinjaman-3-bank',
  'https://www.tribunnews.com/bisnis/2017/11/24/surat-menteri-rini-soemarno-soal-proyek-lrt-jabodetabek-ramai-jadi-perbincangan-investor',
  'https://www.tribunnews.com/metropolitan/2017/08/02/proyek-lrt-terganjal-pembebasan-lahan',
  'https://www.tribunnews.com/metropolitan/2017/08/04/begini-penampakan-lrt-jabodebek-progresnya-sudah-17-persen',
  'https://www.tribunnews.com/metropolitan/2017/12

In [16]:
tribunnews_scraped = False

while tribunnews_scraped is False:
    try:
        urls_tribunnews = tribunnews_scraper(driver)
        tribunnews_scraped = True
    except:
        pass

# The Jakarta Post Section

In [10]:
def thejakartapost_scraper(driver, first_year, second_year, keyword):
    driver.get("https://thejakartapost.com/")

    button = driver.find_element_by_xpath("/html/body/div[4]/div/div[3]/div/div[2]/div[1]/div/div[2]/a[3]")
    button.click()

    input_search = driver.find_element_by_xpath("//*[@id='autocomplete']")
    input_search.send_keys(keyword.lower())
    input_search.send_keys(Keys.RETURN)
    
    try:
        pagination_element = driver.find_element_by_tag_name("div[class*='gsc-cursor']")
        pages = pagination_element.find_elements_by_tag_name("div[class*='gsc-cursor-page']")
    except Exception as e:
        print("No pagination element")
        
        time.sleep(5)
        news_list = driver.find_element_by_xpath("//*[@id='___gcse_0']/div/div/div/div[5]/div[2]/div/div")
        link_list = news_list.find_elements_by_tag_name("a[class*='gs-title']")
        time.sleep(5)
        link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]
        
        pages = None
        
        if bool(link_list) is False:
            return None
        else:
            all_link_list = link_list
    
    if pages is not None:
        all_links_list = []
        for page_idx in tqdm(range(1, len(pages) + 1)):

            if page_idx != 1:
                # Handling element when not loaded
                cannot_click = True
                attempt = 0

                # While element is not clicked
                while cannot_click:
                    try:
                        print("Looking for element.")
                        element_xpath = f"//*[@id='___gcse_0']/div/div/div/div[5]/div[2]/div/div/div[2]/div[11]/div/div[{page_idx}]"

                        # Wait untill element exists
                        element = WebDriverWait(driver, 10).until(
                                lambda driver : driver.find_element_by_xpath(element_xpath)
                        )

                        # Get element 
                        next_page = driver.find_element_by_xpath(element_xpath)

                    except Exception as element_error:

                        print("Element checking failed. Something went wrong.\n", element_error)

                        attempt += 1

                        print(f"Element failed. Repeating process. Currently attempt {attempt}")

                    else:
                        print("Element found. Waiting to be clickable.")
                        try:
                            clickable = WebDriverWait(driver, 10).until(
                                EC.element_to_be_clickable((By.XPATH, element_xpath))
                            )
                            next_page.click()
                            cannot_click = False
                        except Exception as clickable_error:
                            print("Clickalbe checking failed. Something went wrong.\n", clickable_error, end ="\n\n")
                            attempt += 1
                            print(f"Clicking failed. Repeating process. Currently attempt {attempt}")
            time.sleep(5)
            news_list = driver.find_element_by_xpath("//*[@id='___gcse_0']/div/div/div/div[5]/div[2]/div/div")
            link_list = news_list.find_elements_by_tag_name("a[class*='gs-title']")
            time.sleep(5)
            link_list = [link.get_attribute("href") for link in link_list if link.get_attribute("href") is not None]
            urls = list(set(link_list))
            urls = list(filter(None, urls))

            all_links_list.extend(urls)

            pagination_check = driver.find_element_by_tag_name("div[class*='gsc-cursor']")
            pagination_check = pagination_check.find_elements_by_tag_name("div[class*='gsc-cursor-page']")
            if page_idx == len(pagination_check):
                break
            
            
    link_list = all_links_list.copy()
    
    first_year_contents = []
    second_year_contetns = []

    all_index = set(map(link_list.index, link_list))

    error_dates_index = set()

    for link in tqdm(link_list):
        try:
            date = get_date_year(link,"span", "class", "day")
        except Exception as e:
            print(link)
            print(e, end="\n\n")
            error_idx = link_list.index(link)
            error_dates_index.add(error_idx)

    can_get_dates_index = list(all_index - error_dates_index)

    current_date = datetime.datetime.now()
    current_year = current_date.year

    for content_index in tqdm(can_get_dates_index):

        # News Publish Date
        content_date = get_date_year(link_list[content_index],"span", "class", "day")
        print(content_date)

        # First year news
        if content_date >= first_year and content_date < second_year:
            first_year_contents.append(link_list[content_index])
        elif content_date >= second_year and content_date < current_year:
            second_year_contetns.append(link_list[content_index])
        else:
            pass
    return first_year_contents, second_year_contetns

In [79]:
url_thejakartapost = thejakartapost_scraper(driver, 2017, 2018, "ergvervarvarevaerg")

No pagination element


In [80]:
url_thejakartapost

In [18]:
thejakartapost_scraped = False

while thejakartapost_scraped is False:
    try:
        urls_thejakartapost = thejakartapost_scraper(driver)
        thejakartapost_scraped = True
    except:
        pass

# All Scraper

In [11]:
scraper_functions = [
    detik_scraper, liputan6_scraper, kompas_scraper, 
    cnnindo_scraper, tribunnews_scraper, thejakartapost_scraper
]

In [12]:
chromedriver_path = "/mnt/c/Users/Gilang R Ilhami/Desktop/personal_projects/ibm_stuff/chromedriver.exe"
driver = webdriver.Chrome(executable_path=chromedriver_path)

In [13]:
def all_scraper(driver, scraper_list, first_year, second_year, keyword):
    
    all_first_year_contents = []
    all_second_year_contents = []
    
    for scraper in scraper_list:
        urls = scraper(driver, first_year, second_year, keyword)
        
        if urls is not None:
            all_first_year_contents.extend(urls[0])
            all_second_year_contents.extend(urls[1])
        else:
            continue
    return all_first_year_contents, all_second_year_contents

In [36]:
results = all_scraper(driver, scraper_functions, 2017, 2018, "Sawit Sumbermas Sarana")

Last Index -1
Senin, 05 Dec 2016 13:31 WIB


/mnt/c/Users/Gilang R Ilhami/Desktop/personal_projects/ibm_stuff/ibmenv/lib/python3.6/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname WIB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


Rabu, 30 Jan 2019 21:13 WIB
Rabu, 25 Jul 2018 19:02 WIB
Kamis, 25 Jan 2018 16:36 WIB
Senin, 16 Oct 2017 18:07 WIB
Senin, 16 Oct 2017 16:43 WIB
Jumat, 09 Jun 2017 08:55 WIB
Jumat, 13 Jan 2017 22:07 WIB
Jumat, 13 May  2016 20:37 WIB
Jumat, 13 May  2016 10:13 WIB
Kamis, 12 May  2016 09:49 WIB
Rabu, 11 May  2016 20:04 WIB
Jumat, 03 Feb 2017 09:41 WIB
Rabu, 25 Jan 2017 16:01 WIB
Senin, 05 Dec 2016 13:31 WIB
Rabu, 30 Jan 2019 21:13 WIB
2019
Rabu, 25 Jul 2018 19:02 WIB
2018
Kamis, 25 Jan 2018 16:36 WIB
2018
Senin, 16 Oct 2017 18:07 WIB
2017
Senin, 16 Oct 2017 16:43 WIB
2017
Jumat, 09 Jun 2017 08:55 WIB
2017
Jumat, 13 Jan 2017 22:07 WIB
2017
Jumat, 13 May  2016 20:37 WIB
2016
Jumat, 13 May  2016 10:13 WIB
2016
Kamis, 12 May  2016 09:49 WIB
2016
Rabu, 11 May  2016 20:04 WIB
2016
Jumat, 03 Feb 2017 09:41 WIB
2017
Rabu, 25 Jan 2017 16:01 WIB
2017
Senin, 05 Dec 2016 13:31 WIB
2016
Last Index -1
06 Nov 2013, 10:39 WIB
21 Jan 2019, 14:31 WIB
21 Jan 2019, 12:12 WIB
13 Dec 2018, 21:01 WIB
16 Nov 2018,

 12%|█▎        | 1/8 [00:10<01:11, 10.22s/it]

Looking for element.
Element found. Waiting to be clickable.


...

Thu, December 15 2016
2016





 93%|█████████▎| 13/14 [00:04<00:00,  2.62it/s]

Tue, January 13, 2015
2015





100%|██████████| 14/14 [00:05<00:00,  2.57it/s]

Fri, December 5, 2014
2014


In [88]:
len(results)

2

In [23]:
len(all_first_year_contents), len(all_second_year_contents)

(21, 15)

In [84]:
with open('lq45/discovery/all_first_year_contents_MNC.txt', 'w') as f:
    for url in all_first_year_contents:
        f.write("%s\n" % url)

In [85]:
with open('lq45/discovery/all_second_year_contents_MNC.txt', 'w') as f:
    for url in all_second_year_contents:
        f.write("%s\n" % url)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("files/list_lq45_market_structure.csv", index_col=False)

with_social_df = df[df['Social Media Account'] != "zonk"]

import re
companies = with_social_df.Company.values.tolist()
companies = list(map(lambda x : re.sub(r'\([^)]*\)', '', x).replace(".", "").replace("Tbk", "").strip(), companies))

In [3]:
companies

['Adhi Karya',
 'Indika Energy',
 'Jasa Marga',
 'Aneka Tambang',
 'Vale Indonesia',
 'Indika Energy',
 'Bank Central Asia',
 'Bank Negara Indonesia',
 'Bank Rakyat Indonesia',
 'Bank Mandiri',
 'Indofood CBP Sukses Makmur',
 'Indofood Sukses Makmur',
 'Unilever Indonesia',
 'Sri Rejeki Isman',
 'XL Axiata',
 'Media Nusantara Citra',
 'Astra International',
 'Charoen Pokphand Indonesia']

In [39]:
for company in companies:
    company_name = company.lower()
    file_name = "_".join(company_name.split()) + ".txt"
    
    first_year_urls, second_year_urls = all_scraper(driver, scraper_functions, 2017, 2018, company)
    
    with open('lq45/first_year_urls/all_first_year_contents_' + file_name, 'w') as f:
        for url in first_year_urls:
            f.write("%s\n" % url)
    
    with open('lq45/second_year_urls/all_second_year_contents_' + file_name, 'w') as f:
        for url in second_year_urls:
            f.write("%s\n" % url)
    

Last Index -1
Rabu, 31 Jul 2019 17:43 WIB
Getting links...
Rabu, 31 Jul 2019 17:43 WIB
Getting links...
Kamis, 25 Jul 2019 20:01 WIB
Getting links...
Kamis, 11 Jul 2019 22:33 WIB
Getting links...
Selasa, 28 May  2019 19:56 WIB
Getting links...
Senin 29 April 2019, 15:42 WIB
Getting links...
Senin 22 April 2019, 18:25 WIB
Getting links...
Jumat, 05 Apr 2019 10:38 WIB
Getting links...
Jumat, 29 Mar 2019 11:01 WIB
Getting links...
Senin 11 March 2019, 13:49 WIB
Getting links...
Kamis, 21 Feb 2019 18:13 WIB
Getting links...
Jumat, 15 Feb 2019 13:34 WIB
Getting links...
Minggu 03 February 2019, 21:45 WIB
Getting links...
Kamis, 24 Jan 2019 12:13 WIB
Getting links...
Senin, 14 Jan 2019 19:28 WIB
Getting links...
Senin, 14 Jan 2019 15:34 WIB
Getting links...
Selasa 08 January 2019, 16:35 WIB
Getting links...
Senin, 31 Dec 2018 11:50 WIB
Getting links...
Senin 10 December 2018, 18:17 WIB
Getting links...
Selasa, 27 Nov 2018 13:49 WIB
Getting links...
Rabu, 21 Nov 2018 15:54 WIB
Getting links..

Selasa, 26 Feb 2019 22:37 WIB
Senin, 25 Feb 2019 13:47 WIB
Minggu, 24 Feb 2019 17:02 WIB
Kamis, 21 Feb 2019 18:13 WIB
Rabu, 20 Feb 2019 16:17 WIB
Rabu, 20 Feb 2019 14:55 WIB
Selasa, 19 Feb 2019 14:29 WIB
Jumat, 15 Feb 2019 16:47 WIB
Jumat, 15 Feb 2019 15:09 WIB
Jumat, 15 Feb 2019 14:38 WIB
Jumat, 15 Feb 2019 14:20 WIB
Jumat, 15 Feb 2019 14:12 WIB
Jumat, 15 Feb 2019 13:34 WIB
Jumat, 15 Feb 2019 13:08 WIB
Jumat, 15 Feb 2019 11:16 WIB
Jumat, 15 Feb 2019 10:37 WIB
Jumat, 15 Feb 2019 10:11 WIB
Kamis 14 February 2019, 21:07 WIB
Rabu 13 February 2019, 19:20 WIB
Rabu 13 February 2019, 18:51 WIB
Rabu 13 February 2019, 18:40 WIB
Minggu 03 February 2019, 21:45 WIB
Sabtu, 02 Feb 2019 09:55 WIB
Jumat, 01 Feb 2019 18:35 WIB
Jumat, 01 Feb 2019 17:32 WIB
Jumat, 01 Feb 2019 17:02 WIB
Kamis, 31 Jan 2019 18:56 WIB
Kamis, 31 Jan 2019 18:30 WIB
Kamis, 31 Jan 2019 17:02 WIB
Senin, 28 Jan 2019 13:35 WIB
Kamis, 24 Jan 2019 12:13 WIB
Rabu, 23 Jan 2019 15:48 WIB
Selasa, 22 Jan 2019 14:42 WIB
Senin 21 January 20

Jumat, 02 Mar 2018 20:33 WIB
Kamis, 22 Feb 2018 19:03 WIB
Kamis, 22 Feb 2018 17:08 WIB
Kamis, 22 Feb 2018 12:37 WIB
Kamis, 22 Feb 2018 11:51 WIB
Selasa, 20 Feb 2018 21:22 WIB
Selasa, 20 Feb 2018 18:30 WIB
Selasa, 20 Feb 2018 17:27 WIB
Selasa, 20 Feb 2018 13:40 WIB
Senin, 19 Feb 2018 10:46 WIB
Minggu, 18 Feb 2018 09:15 WIB
Sabtu, 17 Feb 2018 19:23 WIB
Sabtu, 17 Feb 2018 17:35 WIB
Sabtu, 17 Feb 2018 17:02 WIB
Sabtu, 17 Feb 2018 16:00 WIB
Sabtu, 17 Feb 2018 14:45 WIB
Sabtu, 17 Feb 2018 14:22 WIB
Sabtu, 17 Feb 2018 13:23 WIB
Sabtu, 17 Feb 2018 11:45 WIB
Kamis, 08 Feb 2018 19:38 WIB
Kamis 08 February 2018, 17:55 WIB
Kamis, 01 Feb 2018 10:58 WIB
Kamis, 01 Feb 2018 10:43 WIB
Kamis, 01 Feb 2018 08:38 WIB
Kamis, 01 Feb 2018 08:11 WIB
Sabtu 27 January 2018, 09:11 WIB
Kamis, 25 Jan 2018 16:36 WIB
Rabu, 24 Jan 2018 16:56 WIB
Rabu, 24 Jan 2018 13:03 WIB
Kamis, 18 Jan 2018 11:21 WIB
Minggu, 14 Jan 2018 13:50 WIB
Sabtu, 13 Jan 2018 16:27 WIB
Sabtu, 13 Jan 2018 14:05 WIB
Jumat, 12 Jan 2018 13:55 WIB
J

KeyboardInterrupt: 

In [21]:
try_list = []
try_list + None

TypeError: can only concatenate list (not "NoneType") to list